<a href="https://colab.research.google.com/github/Troyanovsky/Local-LLM-Comparison-Colab-UI/blob/main/Mixtral_8x7B_Instruct_v0_1_GGUF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mixtral-8x7B-Instruct-v0.1-GGUF WebUI
1. Run the following cell, takes ~5 min
(You may need to confirm to proceed by typing "Y")
2. Click the gradio link at the bottom
3. In Chat settings - Instruction Template: Mistral

```
<s>[INST] {prompt} [/INST]
```

Note:
- this notebook is created at a time when Oobabooga doesn't support Mixtral 8x7B MOE natively
- only run the cell with the quantization you need
- only part of the layers are loaded to GPU so the generation speed will be slow (if you have colab pro, you can adjust the gpu layers to a higher number for faster speed)

In [1]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [1]:
%cd /content
!apt-get -y install -qq aria2

!git clone -b V20231211 https://github.com/Troyanovsky/text-generation-webui
%cd /content/text-generation-webui
!pip install -r requirements.txt
!pip install -U gradio==3.50.2

!conda activate textgen

%cd /content/text-generation-webui/repositories

!git clone https://github.com/abetlen/llama-cpp-python
%cd llama-cpp-python/vendor
!rm -R llama.cpp
!git clone --branch=mixtral https://github.com/ggerganov/llama.cpp.git
%cd ..
!pip list | grep llama
!pip uninstall -y llama_cpp_python
!pip uninstall -y llama_cpp_python_cuda
!CMAKE_ARGS="-DLLAMA_CUDA=on" FORCE_CMAKE=1 pip install . --no-cache-dir

!pip uninstall flash-attn -y
!pip install --no-build-isolation flash-attn==2.3.4

/content
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 120903 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.2_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.2) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.36.0-1_amd64.deb ...
Unpacking aria2 (1.36.0-1) ...
Setting up libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.2) ...
Setting up libaria2-0:amd64 (1.36.0-1) ...
Setting up aria2 (1.36.0-1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.4) ...
/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbb

/bin/bash: line 1: conda: command not found
[Errno 2] No such file or directory: '/content/text-generation-webui/repositories'
/content/text-generation-webui
Cloning into 'llama-cpp-python'...
remote: Enumerating objects: 5590, done.
remote: Counting objects: 100% (1984/1984), done.
remote: Compressing objects: 100% (543/543), done.
remote: Total 5590 (delta 1638), reused 1646 (delta 1411), pack-reused 3606
Receiving objects: 100% (5590/5590), 2.66 MiB | 20.50 MiB/s, done.
Resolving deltas: 100% (3514/3514), done.
/content/text-generation-webui/llama-cpp-python/vendor
Cloning into 'llama.cpp'...
remote: Enumerating objects: 13783, done.
remote: Counting objects: 100% (4277/4277), done.
remote: Compressing objects: 100% (422/422), done.
remote: Total 13783 (delta 4078), reused 3910 (delta 3855), pack-reused 9506
Receiving objects: 100% (13783/13783), 15.86 MiB | 10.67 MiB/s, done.
Resolving deltas: 100% (9623/9623), done.
/content/text-generation-webui/llama-cpp-python
exllama          

In [ ]:
# Q2_K, some on GPU, faster generation (1.68 T/s)
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF/resolve/main/mixtral-8x7b-instruct-v0.1.Q2_K.gguf?download=true -d /content/text-generation-webui/models/ -o mixtral-8x7b-instruct-v0.1.Q2_K.gguf

%cd /content/text-generation-webui
!python server.py --share --n-gpu-layers 20 --n_ctx 2048 --model mixtral-8x7b-instruct-v0.1.Q2_K.gguf

/content/text-generation-webui
2023-12-14 07:38:49 INFO:Loading mixtral-8x7b-instruct-v0.1.Q2_K.gguf...
2023-12-14 07:38:49 INFO:llama.cpp weights detected: models/mixtral-8x7b-instruct-v0.1.Q2_K.gguf
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5
llama_model_loader: loaded meta data with 26 key-value pairs and 995 tensors from models/mixtral-8x7b-instruct-v0.1.Q2_K.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q2_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:          blk.0.ffn_gate.0.weight q2_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    2:          blk.0.ffn_down.0.weight q2_K     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_up.0.weight q2_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    4:      

In [ ]:
# Q4_K_M, half on GPU, very slow generation
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF/resolve/main/mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf?download=true -d /content/text-generation-webui/models/ -o mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf

%cd /content/text-generation-webui
!python server.py --share --n-gpu-layers 12 --n_ctx 2048 --model mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf

In [ ]:
# Q2_K, CPU only, very slow generation (0.03T/s)
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF/resolve/main/mixtral-8x7b-instruct-v0.1.Q2_K.gguf?download=true -d /content/text-generation-webui/models/ -o mixtral-8x7b-instruct-v0.1.Q2_K.gguf

%cd /content/text-generation-webui
!python server.py --share --model mixtral-8x7b-instruct-v0.1.Q2_K.gguf

 *** Download Progress Summary as of Thu Dec 14 07:09:02 2023 *** 
=
[#494504 11GiB/14GiB(77%) CN:16 DL:183MiB ETA:18s]
FILE: /content/text-generation-webui/models//mixtral-8x7b-instruct-v0.1.Q2_K.gguf
-


Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
494504|OK  |   193MiB/s|/content/text-generation-webui/models//mixtral-8x7b-instruct-v0.1.Q2_K.gguf

Status Legend:
(OK):download completed.
/content/text-generation-webui
2023-12-14 07:09:29 INFO:Loading mixtral-8x7b-instruct-v0.1.Q2_K.gguf...
2023-12-14 07:09:29 INFO:llama.cpp weights detected: models/mixtral-8x7b-instruct-v0.1.Q2_K.gguf
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5
llama_model_loader: loaded meta data with 26 key-value pairs and 995 tensors from models/mixtral-8x7b-instruct-v0.1.Q2_K.gguf (version GGUF V3 (lat